# Random Forest for Hate Speech Detection

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from evaluate_classification import EvaluateBinaryClassification

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


### Initialise Random variables

In [2]:
SEED = 123
np.random.seed(SEED)

### Loading Data

In [3]:
BASE = 'D:\\ResearchDataGtx1060\\SentimentData\\Hate\\'
fins_train = ['random_hate_train.csv']
fins_test = ['eastasian_hate_test.csv']
track = 0

In [4]:
# We apply only this preprocessing because our data is already preprocessed
def cleanNonAscii(text):
    '''
    Remove Non ASCII characters from the dataset.
    Arguments:
        text: str
    returns: 
        text: str
    '''
    return ''.join(i for i in text if ord(i) < 128)

In [5]:
df_train = pd.read_csv(BASE+fins_train[track])
df_train.head()

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,ã ¢ â  â ¤ ã ¯ â ¸ â  ã ¢ â  â ¤ ã ¯ â ¸ â ...


In [6]:
df_train.groupby('label').count()

,text
label,
0,2242
1,2242


In [7]:
df_train['text'] = df_train['text'].apply(cleanNonAscii)
df_train.head(5)

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,<hashtag> ...


In [8]:
X_train, y_train = df_train['text'].values, df_train['label'].values

In [9]:
df_test = pd.read_csv(BASE+fins_test[track])
df_test.head()

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [10]:
df_test.groupby('label').count()

,text
label,
0,3898
1,3898


In [11]:
df_test['text'] = df_test['text'].apply(cleanNonAscii)
df_test.head(5)

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [12]:
X_test, y_test = df_test['text'].values, df_test['label'].values

### Transforming data suitable for model format

In [13]:
vectorizer = CountVectorizer()
Xtrain = vectorizer.fit_transform(X_train)
Xtrain = Xtrain.toarray()

In [14]:
Xtest = vectorizer.transform(X_test)
Xtest = Xtest.toarray()

### set hyper parameters to search

In [15]:
param_grid = {
                 'n_estimators': [115, 120, 140, 180],
                 'max_depth': [140]
             }

### Define and fit the model

In [16]:
grid_RF = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=10, scoring='recall', refit='AUC')
grid_RF.fit(Xtrain, y_train)

print("Best parameters set found on development set:")
print()
print(grid_RF.best_params_)
print()
print(grid_RF)

Best parameters set found on development set:

{'max_depth': 140, 'n_estimators': 140}

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [140],
                         'n_estimators': [115, 120, 140, 180]},
             refit='AUC', scoring='recall')


### Evaluating the model with test dataset

In [17]:
actual, predicted = y_test, grid_RF.predict(Xtest)

In [18]:
ebc = EvaluateBinaryClassification(gnd_truths = actual, predictions = predicted)
print(ebc.get_full_report())

EvaluateBinaryClassification Object Created

Total Samples	7796
Positive Samples	3898
Negative Samples	3898
True Positive	2329
True Negative	2123
False Positive	1775
False Negative	1569
Accuracy	0.5710620831195485
Precision	0.567495126705653
Recall	0.5974858902001026
F1 Measure	0.5821044738815296
Cohen Kappa Score	0.14212416623909696
Area Under Curve	0.5710620831195484

              precision    recall  f1-score   support

           0       0.58      0.54      0.56      3898
           1       0.57      0.60      0.58      3898

    accuracy                           0.57      7796
   macro avg       0.57      0.57      0.57      7796
weighted avg       0.57      0.57      0.57      7796



In [19]:
ebc.save_full_report(model_name='Random Forest', path='C:\\Users\\User\\JupyterPythonPredator\\COVID19\\domain_adaptation_rerun_randomhate_eastasianhate_')